# EchoFlow 2.0 - Voice Pathology Detection

**Система диагностики патологий голоса мирового уровня**

---

## 🚀 Быстрый старт

**Этот notebook:**
- ✅ Автоматически настроит окружение
- ✅ Установит все зависимости
- ✅ Загрузит датасет (17.9 ГБ)
- ✅ Обучит модель (~20 часов на GPU T4)
- ✅ Создаст deployment package

**Просто запустите все ячейки!**

---

## ⚙️ Настройки

**Перед запуском:**
1. Runtime → Change runtime type → **GPU T4**
2. Убедитесь, что GPU активирован
3. Запустите все ячейки: Runtime → Run all

---

## 📊 Ожидаемые результаты

- **Accuracy:** 92-95%
- **Sensitivity:** 90-94%
- **Specificity:** 93-96%
- **F1-Score:** 0.91-0.93
- **Время обучения:** ~20 часов на GPU T4

---

## 1️⃣ Проверка GPU

In [ ]:
import torch
import subprocess

print("="*70)
print("  GPU Check")
print("="*70)

if torch.cuda.is_available():
    print(f"✓ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"✓ CUDA Version: {torch.version.cuda}")
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✅ Ready for training!")
else:
    print("⚠ WARNING: No GPU detected!")
    print("Please enable GPU: Runtime → Change runtime type → GPU T4")
    print("Training on CPU will be VERY slow (~200+ hours)")

print("="*70)

## 2️⃣ Клонирование репозитория

In [ ]:
import os

print("="*70)
print("  Cloning Repository")
print("="*70)

# Удалить старую версию если есть
if os.path.exists('/content/Voice'):
    !rm -rf /content/Voice
    print("✓ Removed old version")

# Клонировать репозиторий
!git clone https://github.com/izoomlentoboy-creator/Voice.git /content/Voice

# Перейти в директорию
%cd /content/Voice

print("\n✅ Repository cloned successfully!")
print("="*70)

## 3️⃣ Установка зависимостей

In [ ]:
print("="*70)
print("  Installing Dependencies")
print("="*70)

# Установить системные зависимости для librosa
!apt-get update -qq
!apt-get install -y -qq libsndfile1 ffmpeg

# Установить Python пакеты
!pip install -q transformers librosa soundfile scipy scikit-learn tqdm protobuf sentencepiece

print("\n✅ Dependencies installed successfully!")
print("="*70)

# Проверка установленных пакетов
print("\nInstalled packages:")
import transformers
import librosa
import scipy
import sklearn
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ Transformers: {transformers.__version__}")
print(f"✓ Librosa: {librosa.__version__}")
print(f"✓ SciPy: {scipy.__version__}")
print(f"✓ Scikit-learn: {sklearn.__version__}")

## 4️⃣ Проверка модели

In [ ]:
import sys
sys.path.insert(0, '/content/Voice')

from models.echoflow_v2 import EchoFlowV2, count_parameters

print("="*70)
print("  Model Architecture Check")
print("="*70)

# Создать модель
model = EchoFlowV2(freeze_wav2vec2=True)
params = count_parameters(model)

print(f"\n✓ Total Parameters:    {params['total_millions']:.2f}M")
print(f"✓ Trainable:           {params['trainable_millions']:.2f}M")
print(f"✓ Frozen (Wav2Vec2):   {params['frozen_millions']:.2f}M")

# Тест forward pass
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
audio = torch.randn(2, 48000).to(device)

with torch.no_grad():
    logits = model(audio)
    probs = model.predict_proba(audio)

print(f"\n✓ Forward Pass:        {audio.shape} → {logits.shape}")
print(f"✓ Probabilities:       {probs[0].cpu().tolist()}")
print(f"✓ Device:              {device}")

print("\n✅ Model architecture verified!")
print("="*70)

## 5️⃣ Загрузка датасета

**⚠️ Внимание:** Загрузка займет ~15-20 минут (17.9 ГБ)

In [ ]:
import os
import zipfile
from tqdm import tqdm
import requests

print("="*70)
print("  Dataset Download")
print("="*70)

dataset_dir = '/content/dataset'
dataset_url = 'https://zenodo.org/records/16874898/files/data.zip'
dataset_zip = '/content/data.zip'

# Проверить наличие датасета
if os.path.exists(dataset_dir) and len(os.listdir(dataset_dir)) > 0:
    print("✓ Dataset already exists")
else:
    print("Downloading dataset (17.9 GB)...")
    print("This will take ~15-20 minutes")
    
    # Загрузить с progress bar
    response = requests.get(dataset_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(dataset_zip, 'wb') as f:
        with tqdm(total=total_size, unit='B', unit_scale=True) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))
    
    print("\n✓ Download complete")
    print("\nExtracting dataset...")
    
    # Распаковать
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall('/content/')
    
    # Переименовать
    if os.path.exists('/content/data'):
        os.rename('/content/data', dataset_dir)
    
    # Удалить архив
    os.remove(dataset_zip)
    
    print("✓ Extraction complete")

# Проверить датасет
if os.path.exists(dataset_dir):
    file_count = sum([len(files) for r, d, files in os.walk(dataset_dir)])
    print(f"\n✓ Dataset ready: {file_count} files")
    print("\n✅ Dataset prepared successfully!")
else:
    print("\n⚠ ERROR: Dataset not found!")

print("="*70)

## 6️⃣ Настройка обучения

In [ ]:
# Параметры обучения
EPOCHS = 50
BATCH_SIZE = 16
LEARNING_RATE = 0.0001
SAVE_DIR = '/content/checkpoints'
LOG_FILE = '/content/training.log'

print("="*70)
print("  Training Configuration")
print("="*70)
print(f"Epochs:        {EPOCHS}")
print(f"Batch Size:    {BATCH_SIZE}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Save Dir:      {SAVE_DIR}")
print(f"Log File:      {LOG_FILE}")
print(f"Device:        {'GPU' if torch.cuda.is_available() else 'CPU'}")
print("="*70)

# Создать директории
os.makedirs(SAVE_DIR, exist_ok=True)
print("\n✅ Configuration ready!")

## 7️⃣ Запуск обучения

**⚠️ Внимание:** 
- Обучение займет ~20 часов на GPU T4
- Не закрывайте вкладку браузера
- Colab может отключиться через 12 часов - используйте чекпоинты для продолжения
- Прогресс сохраняется автоматически

In [ ]:
import subprocess
import time
from datetime import datetime

print("="*70)
print("  Starting Training")
print("="*70)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\nTraining will take ~20 hours on GPU T4")
print("Progress will be displayed below...\n")
print("="*70)

# Запустить обучение
!python /content/Voice/train.py \
    --data_dir /content/dataset \
    --batch_size {BATCH_SIZE} \
    --epochs {EPOCHS} \
    --lr {LEARNING_RATE} \
    --save_dir {SAVE_DIR} \
    --log_file {LOG_FILE}

print("\n" + "="*70)
print("  Training Complete!")
print("="*70)
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 8️⃣ Просмотр результатов

In [ ]:
import json
import matplotlib.pyplot as plt

print("="*70)
print("  Training Results")
print("="*70)

# Загрузить историю
history_file = f"{SAVE_DIR}/history.json"
if os.path.exists(history_file):
    with open(history_file, 'r') as f:
        history = json.load(f)
    
    # Вывести лучшие результаты
    best_val_acc = max(history['val_acc'])
    best_epoch = history['val_acc'].index(best_val_acc) + 1
    
    print(f"\nBest Validation Accuracy: {best_val_acc:.2f}%")
    print(f"Best Epoch: {best_epoch}")
    
    # Построить графики
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accuracy
    axes[0, 0].plot(history['train_acc'], label='Train')
    axes[0, 0].plot(history['val_acc'], label='Validation')
    axes[0, 0].set_title('Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy (%)')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Loss
    axes[0, 1].plot(history['train_loss'], label='Train')
    axes[0, 1].plot(history['val_loss'], label='Validation')
    axes[0, 1].set_title('Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Sensitivity
    if 'val_sensitivity' in history:
        axes[1, 0].plot(history['val_sensitivity'])
        axes[1, 0].set_title('Sensitivity')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Sensitivity (%)')
        axes[1, 0].grid(True)
    
    # Specificity
    if 'val_specificity' in history:
        axes[1, 1].plot(history['val_specificity'])
        axes[1, 1].set_title('Specificity')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Specificity (%)')
        axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.savefig('/content/training_results.png', dpi=150)
    plt.show()
    
    print("\n✅ Results plotted successfully!")
else:
    print("\n⚠ History file not found. Training may not be complete.")

print("="*70)

## 9️⃣ Скачивание модели

In [ ]:
import shutil
from google.colab import files

print("="*70)
print("  Creating Deployment Package")
print("="*70)

# Создать deployment package
deployment_dir = '/content/echoflow_v2_deployment'
os.makedirs(deployment_dir, exist_ok=True)

# Скопировать файлы
if os.path.exists(f"{SAVE_DIR}/best.pt"):
    shutil.copy(f"{SAVE_DIR}/best.pt", f"{deployment_dir}/best.pt")
    print("✓ Copied best.pt")

if os.path.exists(f"{SAVE_DIR}/history.json"):
    shutil.copy(f"{SAVE_DIR}/history.json", f"{deployment_dir}/history.json")
    print("✓ Copied history.json")

if os.path.exists(LOG_FILE):
    shutil.copy(LOG_FILE, f"{deployment_dir}/training.log")
    print("✓ Copied training.log")

if os.path.exists('/content/training_results.png'):
    shutil.copy('/content/training_results.png', f"{deployment_dir}/results.png")
    print("✓ Copied results.png")

# Создать архив
shutil.make_archive('/content/echoflow_v2_deployment', 'zip', deployment_dir)
print("\n✓ Created deployment package")

# Показать размер
package_size = os.path.getsize('/content/echoflow_v2_deployment.zip') / (1024**3)
print(f"✓ Package size: {package_size:.2f} GB")

print("\n✅ Deployment package ready!")
print("="*70)

# Скачать
print("\nDownloading deployment package...")
files.download('/content/echoflow_v2_deployment.zip')
print("\n✅ Download started! Check your downloads folder.")

## 🔟 Тестирование модели

In [ ]:
import librosa
import numpy as np

print("="*70)
print("  Model Inference Test")
print("="*70)

# Загрузить обученную модель
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EchoFlowV2().to(device)

if os.path.exists(f"{SAVE_DIR}/best.pt"):
    checkpoint = torch.load(f"{SAVE_DIR}/best.pt", map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    print("✓ Model loaded successfully")
    
    # Найти тестовый аудиофайл
    test_files = []
    for root, dirs, files in os.walk('/content/dataset'):
        for file in files:
            if file.endswith('.wav'):
                test_files.append(os.path.join(root, file))
                if len(test_files) >= 5:
                    break
        if len(test_files) >= 5:
            break
    
    if test_files:
        print(f"\nTesting on {len(test_files)} sample files:\n")
        
        for i, audio_path in enumerate(test_files, 1):
            # Загрузить аудио
            audio, sr = librosa.load(audio_path, sr=16000, mono=True)
            
            # Предсказание
            audio_tensor = torch.FloatTensor(audio).unsqueeze(0).to(device)
            
            with torch.no_grad():
                probs = model.predict_proba(audio_tensor)
                pred = model.predict(audio_tensor)
            
            # Вывести результат
            label = "Pathological" if pred[0] == 1 else "Healthy"
            confidence = probs[0][pred[0]].item() * 100
            
            print(f"{i}. {os.path.basename(audio_path)}")
            print(f"   Prediction: {label} (confidence: {confidence:.1f}%)")
            print(f"   Probs: Healthy={probs[0][0].item():.3f}, Pathological={probs[0][1].item():.3f}\n")
        
        print("✅ Inference test completed!")
    else:
        print("\n⚠ No test files found")
else:
    print("\n⚠ Model checkpoint not found")

print("="*70)

---

## ✅ Готово!

**Что было сделано:**
1. ✅ Настроено окружение
2. ✅ Установлены зависимости
3. ✅ Загружен датасет (17.9 ГБ)
4. ✅ Обучена модель (50 эпох)
5. ✅ Создан deployment package
6. ✅ Протестирована модель

**Результаты:**
- Модель: `/content/checkpoints/best.pt`
- История: `/content/checkpoints/history.json`
- Логи: `/content/training.log`
- Графики: `/content/training_results.png`
- Deployment package: скачан

**Следующие шаги:**
1. Разверните модель на сервере
2. Протестируйте на реальных данных
3. Интегрируйте в приложение

---

## 📚 Документация

- **GitHub:** https://github.com/izoomlentoboy-creator/Voice
- **README:** Полная документация
- **QUICKSTART:** Быстрый старт

---

**Создано с ❤️ для диагностики патологий голоса**